In [1]:
!pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 28.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load and preprocess data
data = pd.read_csv('/work/dataset_full_.csv')
data['python_tokens'] = data['python_tokens'].apply(eval)
data['java_tokens'] = data['java_tokens'].apply(eval)

# Find max length and pad arrays
max_length = max(
    max(len(x) for x in data['python_tokens']),
    max(len(x) for x in data['java_tokens'])
)

# Pad tokens
data['python_tokens'] = data['python_tokens'].apply(
    lambda x: np.pad(x, (0, max_length - len(x)), mode='constant')[:max_length]
)
data['java_tokens'] = data['java_tokens'].apply(
    lambda x: np.pad(x, (0, max_length - len(x)), mode='constant')[:max_length]
)

# absolute matrix
X = np.array([np.abs(np.array(python) - np.array(java)) for python, java in zip(data['python_tokens'], data['java_tokens'])])
y = data['Is_Equal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Define and train model with specified parameters
model = SVC(C=1, gamma='auto', kernel='rbf')
model.fit(X_train, y_train)

# Evaluate
train_accuracy = model.score(X_train, y_train)
test_accuracy = model.score(X_test, y_test)

# Print results
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Train Accuracy: 86.86%
Test Accuracy: 56.25%


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2fe094d0-9fb3-4318-be9c-48cc4ca88a2c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>